In [115]:
import h5py
import json
import numpy as np

In [82]:
f = h5py.File('/Users/valentinwolf/Documents/Studium/Machine_Learning/SoftwareProjekt/example.h5','r')

In [146]:
#Selected games for test set
game_ids_t = []
inputs_t = []
probs_t = []
winners_t = []

# Selected games for validation set
game_ids_v = []
inputs_v = []
probs_v = []
winners_v = []

# probability of a specific game is chosen for the validation set
val_prob = 0.05 
boardsize = 9

i = 0
for data_string in f['json_game_record']:
    if data_string == '':
        break
    json_data = json.loads(data_string)
    id_ = json_data['id']
    
    if id_ in game_ids_t:
        in_val = False
    elif id_ in game_ids_v:
        in_val = True    
    #decide if game will be in validation or train set
    elif np.random.sample(1)[0] > val_prob: 
        game_ids_t.append(id_)
        in_val = False
    else:
        game_ids_v.append(id_)
        in_val = True
        
    if in_val:
        inputs, probs, winners = inputs_v, probs_v, winners_v
    else:
        inputs, probs, winners = inputs_t, probs_t, winners_t

    inputs.append(np.array(json_data['stateprobs']['state']).reshape(12,boardsize,boardsize))
    winners.append(int(json_data['result']))
    probs.append(np.array(jsondatap['stateprobs']['probs']))